In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
from datetime import datetime

In [10]:
import xgboost as xgb

In [5]:
data_folder = 'data'

train = pd.read_csv('/'.join([data_folder, "data2.csv"]), sep = ';')

X = train.drop('cible1', axis = 1)
y = train.cible1

X_test = pd.read_csv('/'.join([data_folder, "test2.csv"]), sep = ';', index_col = 'id')

ids = X_test.index

def rename_X(X):
    return X.rename_axis({'Finess' : 'finess',
                           'Raison sociale' : 'raison_soc',
                           'Provenance des patients (département)' : 'provenance',
                           'Domaines d activités' : 'domain', 
                           'âge (deux classes >75 ans, <= 75 ans)' : 'age',
                           'Nombre de séjours/séances MCO des patients en ALD' : 'n_sejours_ald',
                           'Nombre total de séjours/séances' : 'n_sejours',
                           'annee' : 'annee'},
                          axis = 1)

X = rename_X(X)
X_test = rename_X(X_test)

df = pd.concat([X, X_test], axis=0)

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
print(X.shape)
print(X_test.shape)
print(df.shape)

(1879842, 8)
(664524, 8)
(2544366, 8)


In [7]:
cat_variables = [i for i in df.columns if df[i].dtype == 'O']
print(cat_variables)

from sklearn.preprocessing import LabelEncoder

encods = [LabelEncoder() for col in cat_variables]
for i, col in enumerate(cat_variables):
    df[col] = encods[i].fit_transform(df[col].astype(str))

# encods = [LabelEncoder() for col in cat_variables]

# for i, col in enumerate(cat_variables):
#     X_test[col] = encods[i].fit_transform(X_test[col].astype(str))

num_variables = [i for i in df.columns if df[i].dtype != 'O']
print(num_variables)

df.fillna(0, inplace=True)

X = df[:1879842]
X_test = df[1879842:]

['finess', 'raison_soc', 'provenance', 'domain', 'age']
['finess', 'raison_soc', 'provenance', 'domain', 'age', 'n_sejours_ald', 'n_sejours', 'annee']


In [8]:
print(X.shape)
print(X_test.shape)
print(df.shape)

(1879842, 8)
(664524, 8)
(2544366, 8)


## run `xgboost`

In [11]:
dtrain = xgb.DMatrix(X, y, missing=0.0)
dtest = xgb.DMatrix(X_test)

In [12]:
param = { }
num_round = 100
bst = xgb.train(param, dtrain, num_round)

pred = bst.predict(dtest)

## make submission

In [28]:
id = X_test.index

In [27]:
id.shape

(664524,)

In [48]:
cible = ['{0:.1f}'.format(p) for p in pred]

In [49]:
cible[:20]

['0.0',
 '0.1',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.1',
 '0.0',
 '0.0',
 '-0.0',
 '0.0',
 '0.0',
 '0.1',
 '-0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0']

In [51]:
datename = datetime.now().strftime(format = '%d%m_%H%M')
folder = 'submissions'
filename = folder + '/' + 'xgb_' + datename + '.csv'
print('saving to %s...' % filename)

pd.DataFrame(cible, id).to_csv(filename, sep=';', index_label = 'id', header = ['cible'], float_format = '{.1f}')

saving to submissions/xgb_2909_2133.csv...
